# **TITLE**

## **Import Library**

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy import stats
from scipy.stats import zscore
from imblearn.over_sampling import SMOTE

## **Import Data**

In [5]:
df = pd.read_csv('ibm data.csv')

### Data Type Check (David)


In [13]:
df.sample(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1383,36,No,Non-Travel,1351,Research & Development,9,4,Life Sciences,1,1949,...,2,80,0,5,3,3,5,4,0,2
54,26,No,Travel_Rarely,1443,Sales,23,3,Marketing,1,72,...,3,80,1,5,2,2,2,2,0,0
9,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7
447,35,No,Travel_Rarely,619,Sales,1,3,Marketing,1,600,...,3,80,0,15,2,3,11,9,6,9
828,18,Yes,Non-Travel,247,Research & Development,8,1,Medical,1,1156,...,4,80,0,0,0,3,0,0,0,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

**Note:**  
Datatype masing-masing feature sudah sesuai

### Duplicates Handling (David)

In [22]:
df.duplicated().any()

False

**Note:**  
Data sudah bersih dari nilai duplikat

### Missing Values Handling (Reza)

## **Data Understanding / EDA**

### Descriptive Statistics (Reza)

### Univariate Analysis (Reza)

### Multivariate Analysis (Reza)

### Feature Selection (Gibran)

### Feature Engineering (Gibran)

## **Data Preparation / Preprocessing**

### Data Splitting (Erick)

In [ ]:
X = df.drop('Attrition', axis=1)
y = df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

### Outlier Handling (Erick)

In [ ]:
numerical_cols = X_train.select_dtypes(include=np.number).columns

plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols):
    plt.subplot(5, 6, i + 1)
    sns.boxplot(y=X_train[col])
    plt.title(col)
    plt.tight_layout()
plt.show()

In [ ]:
# Handle Outliers
for col in numerical_cols:
    Q1 = X_train[col].quantile(0.25)
    Q3 = X_train[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    X_train[col] = X_train[col].clip(lower=lower_bound, upper=upper_bound)

In [ ]:
# Check after outlier handling
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols):
    plt.subplot(5, 6, i + 1)
    sns.boxplot(y=X_train[col])
    plt.title(col)
    plt.tight_layout()
plt.show()

### Transformasi (Erick)

#### *Log Transform*

#### *Standardize*

#### *Normalize*

### Encoding (Bila Perlu) (Erick)

### Preprocessing Pipeline (Perlu Discusi dengan Mentor)

### Imbalance Handling (Hanay data Train) (Erick)

## Modeling

### Library For Modeling

### Train (.fit) And Predict (.predict) Baseline Model

### Hyperparameter Tunning

### Final Model

## Model Evaluation

### Test the Prediction of Final Model

### Evaluation Metrics

### Visualitasion Actual vs Prediction (if needed)